In [13]:

import ssl
import pandas as pd
import wandb

ssl._create_default_https_context = ssl._create_unverified_context
import os
from torch.utils.data import DataLoader
from torch import nn
from torchvision.io import read_image
from torchvision.transforms import v2
import torch
import torchvision.transforms as T
import json
import urllib
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision.io import read_image

# Configure notebook parameters

In [14]:
dataset_base_path = '../../datasets/imageBasedModel/zalando'

In [15]:
# Set a device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [21]:
# Get ImageNet labels
imagenet_class_url = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
imagenet_classes = json.loads(urllib.request.urlopen(imagenet_class_url).read())


# # Set a device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extraction_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_lc').eval().to(device)



Using cache found in /Users/romanzberg/.cache/torch/hub/facebookresearch_dinov2_main


In [79]:

#https://www.modelbit.com/blog/deploying-dinov2-for-image-classification-with-modelbit#:~:text=To%20do%20this%2C%20simply%20use,machine%20to%20your%20Colab%20directory.&text=Next%2C%20you'll%20want%20to,ImageNet%20preprocessing%20on%20the%20image.&text=Now%2C%20we%20can%20pass%20the,a%20class%20ID%20and%20label.
def dinov2_classifier(img_path):
    image = read_image(img_path)

    # Preprocess the image
    transform = T.Compose([
        v2.PILToTensor(),
        T.Resize(224, interpolation=T.InterpolationMode.BICUBIC, antialias=True),
        T.CenterCrop(224),
		v2.ConvertImageDtype(torch.float32),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])

    transform =  T.Compose([
        v2.PILToTensor(),
        T.CenterCrop(224),
        v2.ConvertImageDtype(torch.float32),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])


    image = transform(image)




    # Move the image to the GPU if available
    image = image.to(device)

    # Extract the features
    with torch.no_grad():
        features = torch.squeeze(feature_extraction_model(image.unsqueeze(0)))

    # Print the features
    return {'index': features.argmax(-1).item(),
            'label': imagenet_classes[features.argmax(-1).item()]
            }



In [86]:
dinov2_classifier(f'{dataset_base_path}/woman_outfits/-1mcVLVkQDm/0.jpg')

{'index': 638, 'label': 'tights'}